In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime

import sklearn.preprocessing
import sklearn.model_selection

import sklearn.cluster
import umap
import sklearn.neural_network
from sklearn.metrics import plot_confusion_matrix

# Introducción

Se hizo un análisis de comportamiento poblacional en medio de la pandemia de COVID-19, usando los datos provenientes de 'our_world_in_data' y de 'google_mobility'. En particular, como variaba la movilización de las personas hacia distintos tipos de ubicaciones: locales de ventas, farmacias y luegares de mercado, parques, estaciones de transporte, lugares de trabajo y residencias. Usando esta información se buscaba: 1) identificar si los países de Suramérica presentaban patrones discernibles que los diferencien de otras regiones; 2) establecer si hay una relación fuerte entre los patrones de movilización y la tasa de casos en cada país, particularmente en lationamérica.

Se consideró cómo fluctuaban los patrones de movilización diariamente en cada país - en un periodo de dos meses, comprendido entre febrero 14 y abril 17 - y la tasa de casos confirmados - medida como casos nuevos diarios promedio por cada millón de habitantes. Más adelante se muestra el valor de esta tasa para cada país.

In [ ]:
#Importa y adecúa los datos

dataCases = pd.read_csv('../input/uncover/UNCOVER/our_world_in_data/coronavirus-disease-covid-19-statistics-and-research.csv')
dataMobility = pd.read_csv('../input/uncover/UNCOVER/google_mobility/regional-mobility.csv')

dataMobility = dataMobility.sort_values(by=['country','date'])
parciales = dataMobility[dataMobility['region']!='Total'].index
dataMobility = dataMobility.drop(parciales,axis=0).drop(['region'],axis=1)
dataMobility = dataMobility.fillna(0)

dataCases = dataCases.rename(columns = {'location':'country'})
fechasSinCasos = dataCases[dataCases['total_cases']==0].index
dataCases = dataCases.drop(fechasSinCasos,axis=0) # Sólo considera fechas después del primer caso
dataCases = dataCases.sort_values(by=['country','date'])
dataCases = dataCases.drop(['iso_code','total_tests','new_tests','total_tests_per_thousand', 'new_tests_per_thousand', 'tests_units'],axis=1)
dataCases['date'] = pd.to_datetime(dataCases['date'])

In [ ]:
# Omite los países que no se encuentren en ambos datasets

paisesC = dataCases['country'].unique()
paisesM = dataMobility['country'].unique()
paises = []

for pais in paisesM:
    a = dataMobility[dataMobility['country']==pais]
    b = a.shape
    if b[0] < 63: # Remuevo los países que no han reportado alguna fecha
        dataMobility = dataMobility.drop(a.index,axis=0)
        #print('*'+pais)
    elif pais not in paisesC: #Remuevo los países que no reportaron casos
        dataMobility = dataMobility.drop(a.index,axis=0)
        #print('.'+pais)
    else:
        paises.append(pais)

for pais in paisesC:
    a = dataCases[dataCases['country']==pais]
    if pais not in paises: #Remuevo los paises que no reportaron movilidad
        dataCases = dataCases.drop(a.index,axis=0)
        #print(pais)

In [ ]:
# Toma los datos que se usarán

movPaises = []
for pais in paises:
    a = dataMobility[dataMobility['country']==pais]
    datosPais = np.array(a.drop(['country','date'],axis=1))
    movPaises.append(datosPais)

tasaCasos = [] # Tasa promedio (diaria) de casos, por cada millón de habitantes
for pais in paises:
    a = dataCases[dataCases['country']==pais]
    fechas = np.array(a['date'],dtype='datetime64[D]')
    propCasos = np.array(a['total_cases_per_million'])[-1]
    dias = (fechas[-1]-fechas[0]).astype(int)
    tasaCasos.append(propCasos/dias)

In [ ]:
# Reescala y reformatea los datos

movPaises = np.array(movPaises)
X = movPaises.reshape(len(movPaises),-1)
Y = np.array(tasaCasos)

scaler = sklearn.preprocessing.StandardScaler()
X_train = scaler.fit_transform(X)

In [ ]:
plt.figure(figsize=(35,10))
plt.plot(paises,Y)
plt.xticks(paises,rotation=90)
plt.ylabel('Tasa de casos')
plt.grid()
plt.title('Casos nuevos, por día, por millón de habitantes de cada país')
plt.show()

# Visualización de los comportamientos de movilidad de cada país usando UMAP

Se utilizó el algoritmo de UMAP para visualizar los datos de comportamiento en movilización de cada país - que tienen 378 dimensiones - mediante una proyección en 2D, con el fin de identificar si hay comportamientos comunes en latinoamérica. Se realizaron distintas iteraciones para esta proyección, usando distintos valores de 'n_neighbors'. Todas las iteraciones resultaban en agrupaciones parecidas, pero se escogió la que daba una separación más clara entre datos; esa iteración es la que se ilustra a continuación.

In [ ]:
# Visualización reducida por UMAP con clusters por k-means

reducer = umap.UMAP(n_neighbors=15)
reducer.fit(X_train)
embedding = reducer.transform(X_train)

n_clusters = 3
k_means = sklearn.cluster.KMeans(n_clusters=n_clusters)
k_means.fit(embedding)
cluster = k_means.predict(embedding)
distance = k_means.transform(embedding)

In [ ]:
plt.figure(figsize=(15,15))
plt.scatter(embedding[:,0], embedding[:,1], cmap='Paired', s=20.0,c=cluster)
plt.title('Reducción por UMAP de los comportamientos de movilidad')

suramerica = ['Argentina','Bolivia','Brazil','Chile','Colombia','Ecuador','Panama','Paraguay','Peru','Uruguay','Venezuela']
for i in range(len(paises)):   
    x = embedding[:,0][i]
    y = embedding[:,1][i]
    color = 'steelblue'
    size = 'x-small'
    if paises[i] in suramerica:
        color = 'red'
        size = 'small'
    plt.text(x+0.01,y+0.01,paises[i],color=color,fontsize=size)

Como se observa en la gráfica superior, al hacerse una reducción de dimensionalidad sobre los datos de comportamiento en movilidad (usando UMAP) tienden a agruparse muy claramente los paises de Surámerica, lo que se ve reflejado además en la categorización por clusters realizada. En dicho grupo se ubican también países de Centroamérica y el Caribe, además de España y Portugal - entre estos paises conforman una mayoría significativa en este cluster. Asimismo, son pocos los países que resultan en otras ubicaciones de este plano. No parece ser mera coincidencia este comportamiento, puesto que se observa cómo países de otras regiones también se agrupan, como los países bálticos, los paises escandinavos, los paises del golfo árabe, paises del sur de África, etc.

Estos patrones pueden estar reflejando cómo comportamientos culturales comunes entre estos países iberoamericanos (y de cada otra región) definen la reacción de las poblaciones a la pandemia, o pueden ser también producto de la cercanía geográfica y cómo esta influyó en el cronograma de llegada del COVID a dichos países.

# Clasificación según región geográfica usando una red neuronal

Dado el comportamiento observado en la proyección, se buscó generar un algoritmo que pudiera clasificar los países entre latinoamericanos y no-latinoamericanos, basado en estos datos de movilización durante la pandemia. Para esto se utilizó una red neuronal de dos capas, como se observa a continuación, al igual que en el caso anterior, los parámetros se escogieron basado en el mejor comportamiento después de varios intentos variando número de capas, tamaño de capas y función de activación.

In [ ]:
# Clasificación según región

restoLatam = ['Puerto Rico','Mexico','Nicaragua','Guatemala','Honduras','Costa Rica','El Salvador','Dominican Republic','Haiti','Aruba']
Latam = suramerica + restoLatam

Y2 = [1 if i in Latam else 0 for i in paises]
X_train, X_test, Y_train, Y_test = sklearn.model_selection.train_test_split(X, Y2, test_size=0.5)
scaler = sklearn.preprocessing.StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

mlp = sklearn.neural_network.MLPClassifier(activation='relu',hidden_layer_sizes=(63,6),max_iter=10000)
mlp.fit(X_train, Y_train)

Loss = mlp.loss_
F1 =  sklearn.metrics.f1_score(Y_test, mlp.predict(X_test),average='macro')
m = sklearn.metrics.confusion_matrix(Y_test, mlp.predict(X_test))

plot_confusion_matrix(mlp, X_test, Y_test)
plt.title('F1 = {:.2f} ; Loss = {:.2f}'.format(F1,Loss))
plt.xticks([0,1],['Mundo','Latam'])
plt.yticks([1,0],['Latam','Mundo'])
plt.show()

El valor de F1 obtenido y la matriz de confusión correspondiente indican claramente lo que se observó en la representación de UMAP: es completamente posible hacer un modelo que prediga si un país pertenece a Latinoamérica a partir de los comportamientos de movilidad que se dan en la población durante la pandemia.

# Predicción de la tasa de infectados a partir del comportamiento en movilización

Para finalizar, se utilizó otra vez una red neuronal, esta vez para identificar relaciones entre los patrones de movilización y la tasa de infecciones que se da en cada país. Para hacer de este un problema de clasificación, se dividieron los paises en dos grupos: los de alta tasa de infectados y los de baja tasa, asignando a cada categoría la mitad de los datos. Se eligió esta manera de dividir con base en lo observado en la gráfica de tasas de infección que se encuentra arriba: hay muchos países (aproximadamente la mitad) cuyas tasas de infección apenas alcanzan valores de 10, mientras que el resto tiene tasas que oscilan entre 20 y 100 infecciones diarias por millón de habitantes.

In [ ]:
Y3 = 1*(Y>np.sort(Y)[int(len(Y)/2)])

X_train, X_test, Y_train, Y_test = sklearn.model_selection.train_test_split(X, Y3, test_size=0.7)
scaler = sklearn.preprocessing.StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

mlp = sklearn.neural_network.MLPClassifier(activation='logistic',hidden_layer_sizes=(7,9,6),max_iter=10000)
mlp.fit(X_train, Y_train)

Loss = mlp.loss_
F1 =  sklearn.metrics.f1_score(Y_test, mlp.predict(X_test),average='macro')
m = sklearn.metrics.confusion_matrix(Y_test, mlp.predict(X_test))

plot_confusion_matrix(mlp, X_test, Y_test)
plt.title('F1 = {:.2f} ; Loss = {:.2f}'.format(F1,Loss))
plt.xticks([0,1],['Bajo','Alto'])
plt.yticks([1,0],['Alto','Bajo'])
plt.show()

Se puede ver que el modelo obtenido es bastante confiable, con un valor de F1 de 0.76.

### Aplicación del modelo sobre paises latinoamericanos

Una vez se tuvo el modelo entrenado con los datos globales. Se procedió a probarlo sobre los datos de los paises de latinoamerica. 

In [ ]:
Y_lat = [Y[i] for i in range(len(paises)) if paises[i] in Latam ]
Y_lat = 1*(Y_lat>np.sort(Y)[int(len(Y)/2)])
X_lat = [X[i] for i in range(len(paises)) if paises[i] in Latam ]

Loss = mlp.loss_
F1 =  sklearn.metrics.f1_score(Y_lat, mlp.predict(X_lat),average='macro')
m = sklearn.metrics.confusion_matrix(Y_lat, mlp.predict(X_lat))

plot_confusion_matrix(mlp, X_lat, Y_lat)
plt.title('F1 = {:.2f} ; Loss = {:.2f}'.format(F1,Loss))
plt.xticks([0,1],['Bajo','Alto'])
plt.yticks([1,0],['Alto','Bajo'])
plt.show()

Como se observa del puntaje F1, esta relación entre comportamientos en movilización y tasa de infectados parece ser mucho más debil - o incluso inexistente - en el caso de los países suramericanos. Cabe resaltar que el modelo ha predicho que todos los paises de latinoamerica deberían estar en la mitad 'alta' de tasa de coronavirus, a pesar de que sólo la mitad pertenece a esa categoría. Esto da lugar a dos posibilidades: 1) los comportamientos que en otros paises implicarían un alto número de casos de coronavirus no tienen la misma correlación para latinoamérica, ó 2) los datos de latinoamérica no son tan confiables como los del resto del mundo